In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.formula.api import ols
from scipy import stats
plt.rcParams['figure.figsize'] = [16, 12]
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("Boston.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

Kroki postepowania:

1. Dzielimy zbiór na treningowy i testowy
2. Na zbiorze treningowym uczymy model regresji zmiennej lstat do medv
3. Wyliczamy na zbiorze testowym predykcje i na ich podstawie porównując do wartości rzeczywisych wyliczamy MSE
4. Na zbiorze treningowym stosujemy GridSearchCV, który odpowie nam na pytanie który stopień wielomianu jest najlepszy
5. Na zbiorze treningowym uczymy model wielomianu odpowiedniego stopnia 
6. Na zbiorze testowym sprawdzamy MSE tego modelu jak w przypadku modelu zwykłej prostej regresji
7. Porówujemy oba wyniki i wskazujemy lepszy model.


In [ ]:
X = df.lstat.values.reshape(-1,1)
y = df.medv.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)


In [ ]:
y_pred_lr = model_lr.predict(X_test)


In [ ]:
mse = mean_squared_error(y_test, y_pred_lr)
print('MSE: ', mse)

In [ ]:
poly_regress_pipeline = make_pipeline(PolynomialFeatures(), LinearRegression())

random_search = GridSearchCV(poly_regress_pipeline, {
    'polynomialfeatures__degree': range(1, 10)
}, cv=3, verbose=1)

random_search.fit(X_train.reshape(-1,1), y_train)
random_search.best_params_

In [ ]:
best_model  = make_pipeline(PolynomialFeatures(5), LinearRegression()).fit(X_test.reshape(-1,1), y_test)
y_pred_grid = best_model.predict(X_test.reshape(-1,1))
y_test.shape, y_pred_grid.shape, y_pred_lr.shape, X_test.shape

In [ ]:
mse_grid = mean_squared_error(y_test, y_pred_grid)
mse_grid

In [ ]:
# wykresy

plt.scatter(X_test, y_test, facecolors='None', edgecolors='k', alpha=.7) 
sns.regplot(X_test, y_pred_lr, ci=None, label='Linear', scatter=False, color='grey')
sns.regplot(X_test, y_pred_grid, ci=None, label='Polynomial', order=5, scatter=False, color='pink')
plt.legend()



In [ ]:
print('MSE przy zastosowaniu regresji liniowej wyniosl:', mse)
print('MSE po zastosowaniu GridSearchCV wyniosl:', mse_grid)
print('Lepszy jest model po zastosowaniu GridSearchCV')